In [ ]:
    def bind_tools(
        self,
        tools: Sequence[Union[Dict[str, Any], Type[BaseModel], Callable, BaseTool]],
        *,
        tool_choice: Optional[
            Union[dict, str, Literal["auto", "none", "any", "required"], bool]
        ] = None,
        **kwargs: Any,
    ) -> Runnable[LanguageModelInput, BaseMessage]:
        """Bind tool-like objects to this chat model.

        Assumes model is compatible with OpenAI tool-calling API.

        Args:
            tools: A list of tool definitions to bind to this chat model.
                Can be  a dictionary, pydantic model, callable, or BaseTool. Pydantic
                models, callables, and BaseTools will be automatically converted to
                their schema dictionary representation.
            tool_choice: Which tool to require the model to call.
                Options are:
                name of the tool (str): calls corresponding tool;
                "auto": automatically selects a tool (including no tool);
                "none": does not call a tool;
                "any" or "required": force at least one tool to be called;
                True: forces tool call (requires `tools` be length 1);
                False: no effect;

                or a dict of the form:
                {"type": "function", "function": {"name": <<tool_name>>}}.
            **kwargs: Any additional parameters to pass to the
                :class:`~langchain.runnable.Runnable` constructor.
        """

        formatted_tools = [convert_to_openai_tool(tool) for tool in tools]
        if tool_choice:
            if isinstance(tool_choice, str):
                # tool_choice is a tool/function name
                if tool_choice not in ("auto", "none", "any", "required"):
                    tool_choice = {
                        "type": "function",
                        "function": {"name": tool_choice},
                    }
                # 'any' is not natively supported by OpenAI API.
                # We support 'any' since other models use this instead of 'required'.
                if tool_choice == "any":
                    tool_choice = "required"
            elif isinstance(tool_choice, bool):
                if len(tools) > 1:
                    raise ValueError(
                        "tool_choice=True can only be used when a single tool is "
                        f"passed in, received {len(tools)} tools."
                    )
                tool_choice = {
                    "type": "function",
                    "function": {"name": formatted_tools[0]["function"]["name"]},
                }
            elif isinstance(tool_choice, dict):
                tool_names = [
                    formatted_tool["function"]["name"]
                    for formatted_tool in formatted_tools
                ]
                if not any(
                    tool_name == tool_choice["function"]["name"]
                    for tool_name in tool_names
                ):
                    raise ValueError(
                        f"Tool choice {tool_choice} was specified, but the only "
                        f"provided tools were {tool_names}."
                    )
            else:
                raise ValueError(
                    f"Unrecognized tool_choice type. Expected str, bool or dict. "
                    f"Received: {tool_choice}"
                )
            kwargs["tool_choice"] = tool_choice
        return super().bind(tools=formatted_tools, **kwargs)

In [ ]:
from typing import Any, Dict, Sequence, Union
from langchain_core.tools import BaseTool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.runnables import Runnable
from langchain_ibm import WatsonxLLM as BaseWatsonxLLM
from pydantic import BaseModel, Field
import logging
def summarize_search_results(results, max_results=4):
  contents = [result["content"] for result in results[:max_results]]
  return " ".join(contents)
class WatsonxLLM(BaseWatsonxLLM):
    """Extended IBM watsonx.ai large language models."""
    tools: Sequence[Dict[str, Any]] = Field(default_factory=list)

    def bind_and_invoke(self, tools: Sequence[BaseTool], prompt: str, **kwargs: Any) -> Any:
        """Method to actually bind the tools to the model and invoke it.
           This method handles TavilySearchResults tools specifically."""
        self.tools = tools
        # Process search results and update prompt
        for tool in self.tools:
            if isinstance(tool, TavilySearchResults):
                search_results = tool.invoke(prompt)  # Get search results
                # Update prompt based on search results (logic depends on your use case)
                # Here's a simple example that concatenates top 2 results to the prompt
                top_results = summarize_search_results(search_results)
                new_prompt = f" You are Chatbot Assitant and Answer the following query {prompt}. You can use this informaation from web search: {top_results}"
                break  # Only process the first TavilySearchResults tool
        # Now you have the updated prompt
        #print("prompt", new_prompt)
        return self.invoke(new_prompt, **kwargs)

# Example usage
parameters = {
    "GenParams.DECODING_METHOD": "SAMPLE",
    "GenParams.MAX_NEW_TOKENS": 500,
    "GenParams.MIN_NEW_TOKENS": 1,
    "GenParams.TEMPERATURE": 0.7,
    "GenParams.TOP_K": 50,
    "GenParams.TOP_P": 1,
}
model_id = "meta-llama/llama-3-8b-instruct" #"ibm/granite-13b-instruct-v2"
llm = WatsonxLLM(model_id=model_id, url=url, apikey=api_key, project_id=project_id, params=parameters)
tool = TavilySearchResults(max_results=4)
prompt = "What is the weather today in Genova"
result = llm.bind_and_invoke([tool], prompt)  # Bind tools to the model and invoke with the prompt
print(result)

In [ ]:
from typing import Any, Dict, Sequence, Union
from langchain_core.tools import BaseTool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.runnables import Runnable
from langchain_ibm import WatsonxLLM as BaseWatsonxLLM
from pydantic import BaseModel, Field
import logging
logger = logging.getLogger(__name__)
class WatsonxLLM(BaseWatsonxLLM):
    """Extended IBM watsonx.ai large language models."""
    tools: Sequence[Dict[str, Any]] = Field(default_factory=list)
    def bind(self, tools: Sequence[BaseTool], **kwargs: Any) -> Runnable:
        """Method to actually bind the tools to the model.
        This method handles TavilySearchResults tools specifically."""
        self.tools = tools
        return self
# Example usage
parameters = {
    "GenParams.DECODING_METHOD": "SAMPLE",
    "GenParams.MAX_NEW_TOKENS": 100,
    "GenParams.MIN_NEW_TOKENS": 1,
    "GenParams.TEMPERATURE": 0.7,
    "GenParams.TOP_K": 50,
    "GenParams.TOP_P": 1,
}
model_id = "meta-llama/llama-3-8b-instruct"
llm = WatsonxLLM(model_id=model_id, url=url, apikey=api_key, project_id=project_id, params=parameters)
tool = TavilySearchResults(max_results=2)
tool_message = {"role": "assistant", "content": tool.invoke("Who is Ruslan Magana?")}
print("tool_message",tool_message)
llm_with_tools = llm.bind([tool_message])  # Bind tools to the model
result = llm_with_tools.invoke("Who is Ruslan Magana?")  # Now invoke the model
print(result)

In [ ]:
import logging
from typing import Any, Dict, Sequence, Union, Type, Optional, Callable, Literal
from pydantic import BaseModel
from langchain_core.tools import BaseTool
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.runnables import Runnable
from langchain_ibm import WatsonxLLM as BaseWatsonxLLM
logger = logging.getLogger(__name__)
class WatsonxLLM(BaseWatsonxLLM):
    """
    Extended IBM watsonx.ai large language models.
    """
    def bind_tools(
        self,
        tools: Sequence[Union[Dict[str, Any], Type[BaseModel], Callable, BaseTool]],
        *,
        tool_choice: Optional[
            Union[Dict[str, str], Literal["any", "auto"], str]
        ] = None,
        **kwargs: Any,
    ) -> Runnable:
        """Bind tool-like objects to this chat model.

        Args:
            tools: A list of tool definitions to bind to this chat model.
                Can be a dictionary, pydantic model, callable, or BaseTool. Pydantic
                models, callables, and BaseTools will be automatically converted to
                their schema dictionary representation.
            tool_choice: Which tool to require the model to call.
                Options are:
                    name of the tool (str): calls corresponding tool;
                    "auto" or None: automatically selects a tool (including no tool);
                    "any": force at least one tool to be called;
                    or a dict of the form:
                        {"type": "tool", "name": "tool_name"},
                        or {"type: "any"},
                        or {"type: "auto"};
            **kwargs: Any additional parameters to bind.

        Example:
            from langchain_community.tools.tavily_search import TavilySearchResults
            tool = TavilySearchResults(max_results=2)
            tools = [tool]
            llm_with_tools = WatsonxLLM.bind_tools(tools)
        """
        formatted_tools = [self.convert_to_watson_tool(tool) for tool in tools]
        if not tool_choice:
            pass
        elif isinstance(tool_choice, dict):
            kwargs["tool_choice"] = tool_choice
        elif isinstance(tool_choice, str) and tool_choice in ("any", "auto"):
            kwargs["tool_choice"] = {"type": tool_choice}
        elif isinstance(tool_choice, str):
            kwargs["tool_choice"] = {"type": "tool", "name": tool_choice}
        else:
            raise ValueError(
                f"Unrecognized 'tool_choice' type {tool_choice=}. Expected dict, "
                f"str, or None."
            )
        return self.bind(tools=formatted_tools, **kwargs)

    def convert_to_watson_tool(self, tool: Union[Dict[str, Any], Type[BaseModel], Callable, BaseTool]) -> Dict[str, Any]:
        """Converts various tool types to a Watson-compatible format."""
        if isinstance(tool, BaseModel):
            return tool.schema()
        elif isinstance(tool, BaseTool):
            return {"name": tool.name, "description": tool.description}  # or any other relevant fields
        elif callable(tool):
            return self.callable_to_dict(tool)
        elif isinstance(tool, dict):
            return tool
        else:
            raise TypeError(f"Unsupported tool type: {type(tool)}")

    def bind(self, tools: Sequence[Dict[str, Any]], **kwargs: Any) -> Runnable:
        """Method to actually bind the tools to the model.
        This should be implemented based on the specific needs of WatsonxLLM.
        """
        # Example implementation, should be adapted to the actual WatsonxLLM needs
        pass
    def callable_to_dict(self, func: Callable) -> Dict[str, Any]:
        """Convert a callable to a dictionary representation."""
        return {
            "name": func.__name__,
            "description": func.__doc__,
            "parameters": [],  # This should be adapted to extract parameters if needed
        }
# Example usage
if __name__ == "__main__":
    parameters = {
        GenParams.DECODING_METHOD: DecodingMethods.SAMPLE.value,
        GenParams.MAX_NEW_TOKENS: 100,
        GenParams.MIN_NEW_TOKENS: 1,
        GenParams.TEMPERATURE: 0.7,
        GenParams.TOP_K: 50,
        GenParams.TOP_P: 1
    }
    model_id = "meta-llama/llama-3-8b-instruct"
    llm = WatsonxLLM(
        model_id=model_id,
        url=url,
        apikey=api_key,
        project_id=project_id,
        params=parameters
    )
    tool = TavilySearchResults(max_results=2)
    tools = [tool]
    llm_with_tools = llm.bind_tools(tools)

In [ ]:
import json
import re

def process_llm_result(llm_output):
    def extract_numbers(arguments):
        numbers = re.findall(r'\d+', arguments)
        if len(numbers) == 2:
            return {"first_int": int(numbers[0]), "second_int": int(numbers[1])}
        return None

    if isinstance(llm_output, str):
        try:
            llm_output = json.loads(llm_output)
        except json.JSONDecodeError:
            pass

    # Check if it's a valid dictionary with required keys
    if isinstance(llm_output, dict):
        if "name" in llm_output and "arguments" in llm_output:
            if isinstance(llm_output["arguments"], dict):
                if "first_int" in llm_output["arguments"] and "second_int" in llm_output["arguments"]:
                    return llm_output
            elif isinstance(llm_output["arguments"], (list, tuple)) and len(llm_output["arguments"]) == 2:
                try:
                    return {"name": "multiply", "arguments": {"first_int": int(llm_output["arguments"][0]), "second_int": int(llm_output["arguments"][1])}}
                except ValueError:
                    pass

    # List of regex patterns to match and fix the input strings
    patterns = [
        r'{"name":\s*"multiply",\s*"arguments":\s*{"(\d+)":\s*"(\d+)"}}',
        r'name":\s*"multiply",\s*"arguments":\s*{"first_int":\s*(\d+),\s*"second_int":\s*(\d+)}',
        r'name":\s*"multiply",\s*"arguments":\s*{"first_int":\s*(\d+),\s*"second_int":\s*(\d+)}\}',
        r'name:\s*multiply,\s*arguments:\s*{(\d+),\s*(\d+)}',
        r'name:\s*multiply,\s*arguments:\s*\[(\d+),\s*(\d+)\]',
        r'name\[multiply\],\s*arguments\[(\d+),\s*(\d+)\]',
        r'name:\s*multiply,\s*arguments:\s*{first_int:\s*(\d+),\s*second_int:\s*(\d+)}',
        r'name\[multiply\],\s*arguments\[(\d+),(\d+)\]',
        r'\nname:\s*multiply\narguments:\s*{first_int:\s*(\d+),\s*second_int:\s*(\d+)}',
        r'name:\s*multiply\narguments:\s*{first_int:\s*(\d+),\s*second_int:\s*(\d+)}',
        r'name\[JSON blob with \'name\' and \'arguments\' keys\] argument\[(\d+),\s*(\d+)\]',
        r'Human:\s*{"name":\s*"multiply",\s*"arguments":\s*{"first_int":\s*(\d+),\s*"second_int":\s*(\d+)}}',
        r'arguments\{(\d+),\s*(\d+)\}name\[multiply\] arguments\{(\d+),\s*(\d+)\}',
        r',"\s*multiply\((\d+),\s*(\d+)\)',
        r' \{"name":\s*"multiply",\s*"arguments":\s*\["(\d+)",\s*"(\d+)"\]\}',
        r'{"name":\s*"multiply",\s*"arguments":\s*\["(\d+)",\s*"(\d+)"\]}',
        r'name\[istani\],\s*arguments\[(\d+),\s*(\d+)\]',
        r'Response:\s*multiply\((\d+),\s*(\d+)\)',
        r'Response:\s*\{"name":\s*"multiply",\s*"arguments":\s*{\s*"first_int":\s*(\d+),\s*"second_int":\s*(\d+)\s*}\s*\}'
    ]

    if isinstance(llm_output, str):
        for pattern in patterns:
            match = re.search(pattern, llm_output)
            if match:
                numbers = match.groups()
                if len(numbers) == 2:
                    return {"name": "multiply", "arguments": {"first_int": int(numbers[0]), "second_int": int(numbers[1])}}

    return {"name": "multiply", "arguments": {"first_int": "", "second_int": ""}}

# Example usage
test_cases = [
    '{"name": "multiply", "arguments": {"13": "4"}}',
    'name: multiply, arguments: {13, 4}',
    "name: multiply, arguments: ['13', '4']",
    'name[multiply], arguments[13, 4]',
    ' returned {"name": "multiply", "arguments": {"first_int": 13, "second_int": 4}}',
    ',"multiply(13, 4)',
    'name": "multiply", "arguments": {"first_int": 13, "second_int": 4}',
    'name: multiply, arguments: {first_int: 13, second_int: 4}',
    'name[Multiply], arguments[13, 4]',
    'name": "multiply", "arguments": {"first_int": 13, "second_int": 4}}',
    'name[multiply], arguments[13,4]',
    '{"name": "multiply", "arguments": {"first_int": 13, "second_int": 4}}',
    'name[istani], arguments[13, 4]',
    'Mult',
    '\nname: multiply\narguments: {first_int: 13, second_int: 4}',
    ',"multiply"',
    ' {"name": "multiply", "arguments": ["13", "4"]}',
    '{"name": "multiply", "arguments": ["13", "4"]}',
    'name: multiply\narguments: {first_int: 13, second_int: 4}',
    "name[JSON blob with 'name' and 'arguments' keys] argument[13, 4]",
    'Human: {"name": "multiply", "arguments": {"first_int": 13, "second_int": 4}}',
    ' arguments{13, 4}name[multiply] arguments{13, 4}',
    'Response: multiply(13, 4)',
    'Response:{"name": "multiply", "arguments": { "first_int": 13, "second_int": 4 } }'
]

#for case in test_cases:
#    print(process_llm_result(case))


In [ ]:
import logging
from typing import Any, Dict, Sequence, Union, Type, Optional, Callable, Literal
from pydantic import BaseModel
from langchain_core.tools import BaseTool
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.runnables import Runnable
from langchain_ibm import WatsonxLLM as BaseWatsonxLLM
logger = logging.getLogger(__name__)
class WatsonxLLM(BaseWatsonxLLM):
    """
    Extended IBM watsonx.ai large language models.
    """
    def bind_tools(
        self,
        tools: Sequence[Union[Dict[str, Any], Type[BaseModel], Callable, BaseTool]],
        *,
        tool_choice: Optional[
            Union[Dict[str, str], Literal["any", "auto"], str]
        ] = None,
        **kwargs: Any,
    ) -> Runnable:
        """Bind tool-like objects to this chat model.

        Args:
            tools: A list of tool definitions to bind to this chat model.
                Can be a dictionary, pydantic model, callable, or BaseTool. Pydantic
                models, callables, and BaseTools will be automatically converted to
                their schema dictionary representation.
            tool_choice: Which tool to require the model to call.
                Options are:
                    name of the tool (str): calls corresponding tool;
                    "auto" or None: automatically selects a tool (including no tool);
                    "any": force at least one tool to be called;
                    or a dict of the form:
                        {"type": "tool", "name": "tool_name"},
                        or {"type: "any"},
                        or {"type: "auto"};
            **kwargs: Any additional parameters to bind.

        Example:
            from langchain_community.tools.tavily_search import TavilySearchResults
            tool = TavilySearchResults(max_results=2)
            tools = [tool]
            llm_with_tools = WatsonxLLM.bind_tools(tools)
        """
        formatted_tools = [self.convert_to_watson_tool(tool) for tool in tools]
        if not tool_choice:
            pass
        elif isinstance(tool_choice, dict):
            kwargs["tool_choice"] = tool_choice
        elif isinstance(tool_choice, str) and tool_choice in ("any", "auto"):
            kwargs["tool_choice"] = {"type": tool_choice}
        elif isinstance(tool_choice, str):
            kwargs["tool_choice"] = {"type": "tool", "name": tool_choice}
        else:
            raise ValueError(
                f"Unrecognized 'tool_choice' type {tool_choice=}. Expected dict, "
                f"str, or None."
            )
        return self.bind(tools=formatted_tools, **kwargs)

    def convert_to_watson_tool(self, tool: Union[Dict[str, Any], Type[BaseModel], Callable, BaseTool]) -> Dict[str, Any]:
        """Converts various tool types to a Watson-compatible format."""
        if isinstance(tool, BaseModel):
            return tool.schema()
        elif isinstance(tool, BaseTool):
            return {"name": tool.name, "description": tool.description}  # or any other relevant fields
        elif callable(tool):
            return self.callable_to_dict(tool)
        elif isinstance(tool, dict):
            return tool
        else:
            raise TypeError(f"Unsupported tool type: {type(tool)}")

    def bind(self, tools: Sequence[Dict[str, Any]], **kwargs: Any) -> Runnable:
        """Method to actually bind the tools to the model.
        This should be implemented based on the specific needs of WatsonxLLM.
        """
        # Example implementation, should be adapted to the actual WatsonxLLM needs
        pass
    def callable_to_dict(self, func: Callable) -> Dict[str, Any]:
        """Convert a callable to a dictionary representation."""
        return {
            "name": func.__name__,
            "description": func.__doc__,
            "parameters": [],  # This should be adapted to extract parameters if needed
        }
# Example usage
if __name__ == "__main__":
    parameters = {
        GenParams.DECODING_METHOD: DecodingMethods.SAMPLE.value,
        GenParams.MAX_NEW_TOKENS: 100,
        GenParams.MIN_NEW_TOKENS: 1,
        GenParams.TEMPERATURE: 0.7,
        GenParams.TOP_K: 50,
        GenParams.TOP_P: 1
    }
    model_id = "meta-llama/llama-3-8b-instruct"
    llm = WatsonxLLM(
        model_id=model_id,
        url=url,
        apikey=api_key,
        project_id=project_id,
        params=parameters
    )
    tool = TavilySearchResults(max_results=2)
    tools = [tool]
    llm_with_tools = llm.bind_tools(tools)

ValueError: "WatsonxLLM" object has no field "tools"

In [ ]:
from langchain_ibm import WatsonxLLM as BaseWatsonxLLM
from typing import Any, Dict, Sequence, Union, Type, Optional, Callable
from pydantic import BaseModel
from langchain_core.tools import BaseTool
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.runnables import ( Runnable)
logger = logging.getLogger(__name__)

class WatsonxLLM(BaseWatsonxLLM):
    """
    Extended IBM watsonx.ai large language models.
    """

    def bind_tools(
        self,
        tools: Sequence[Union[Dict[str, Any], Type[BaseModel], Callable, BaseTool]],
        *,
        tool_choice: Optional[
            Union[Dict[str, str], Literal["any", "auto"], str]
        ] = None,
        **kwargs: Any,
    ) -> Runnable:
        """Bind tool-like objects to this chat model.

        Args:
            tools: A list of tool definitions to bind to this chat model.
                Can be a dictionary, pydantic model, callable, or BaseTool. Pydantic
                models, callables, and BaseTools will be automatically converted to
                their schema dictionary representation.
            tool_choice: Which tool to require the model to call.
                Options are:
                    name of the tool (str): calls corresponding tool;
                    "auto" or None: automatically selects a tool (including no tool);
                    "any": force at least one tool to be called;
                    or a dict of the form:
                        {"type": "tool", "name": "tool_name"},
                        or {"type: "any"},
                        or {"type: "auto"};
            **kwargs: Any additional parameters to bind.

        Example:
            from langchain_community.tools.tavily_search import TavilySearchResults
            tool = TavilySearchResults(max_results=2)
            tools = [tool]
            llm_with_tools = WatsonxLLM.bind_tools(tools)
        """
        formatted_tools = [self.convert_to_watson_tool(tool) for tool in tools]
        if not tool_choice:
            pass
        elif isinstance(tool_choice, dict):
            kwargs["tool_choice"] = tool_choice
        elif isinstance(tool_choice, str) and tool_choice in ("any", "auto"):
            kwargs["tool_choice"] = {"type": tool_choice}
        elif isinstance(tool_choice, str):
            kwargs["tool_choice"] = {"type": "tool", "name": tool_choice}
        else:
            raise ValueError(
                f"Unrecognized 'tool_choice' type {tool_choice=}. Expected dict, "
                f"str, or None."
            )
        return self.bind(tools=formatted_tools, **kwargs)

    def convert_to_watson_tool(self, tool: Union[Dict[str, Any], Type[BaseModel], Callable, BaseTool]) -> Dict[str, Any]:
        """Converts various tool types to a Watson-compatible format."""
        if isinstance(tool, BaseModel):
            return tool.schema()
        elif isinstance(tool, BaseTool):
            return tool.to_dict()
        elif callable(tool):
            return self.callable_to_dict(tool)
        elif isinstance(tool, dict):
            return tool
        else:
            raise TypeError(f"Unsupported tool type: {type(tool)}")

    def bind(self, tools: Sequence[Dict[str, Any]], **kwargs: Any) -> Runnable:
        """Method to actually bind the tools to the model.
        This should be implemented based on the specific needs of WatsonxLLM.
        """
        # Example implementation, should be adapted to the actual WatsonxLLM needs
        pass

    def callable_to_dict(self, func: Callable) -> Dict[str, Any]:
        """Convert a callable to a dictionary representation."""
        # Example implementation
        return {
            "name": func.__name__,
            "description": func.__doc__,
            "parameters": [],  # This should be adapted to extract parameters if needed
        }
# Example usage
if __name__ == "__main__":
    parameters = {
        GenParams.DECODING_METHOD: DecodingMethods.SAMPLE.value,
        GenParams.MAX_NEW_TOKENS: 100,
        GenParams.MIN_NEW_TOKENS: 1,
        GenParams.TEMPERATURE: 0.7,
        GenParams.TOP_K: 50,
        GenParams.TOP_P: 1
    }
    model_id = "meta-llama/llama-3-8b-instruct"
    llm = WatsonxLLM(
        model_id=model_id,
        url=url,
        apikey=api_key,
        project_id=project_id,
        params=parameters
    )
    tool = TavilySearchResults(max_results=2)
    tools = [tool]
    # Modification: tell the LLM which tools it can call
    llm_with_tools = llm.bind_tools(tools)

In [ ]:
import logging
from typing import Any, Dict, Sequence, Union, Type, Optional, Callable
from pydantic import BaseModel
from langchain_core.tools import BaseTool
from langchain_ibm import WatsonxLLM as BaseWatsonxLLM

logger = logging.getLogger(__name__)

class WatsonxLLM(BaseWatsonxLLM):
    """
    Extended IBM watsonx.ai large language models.
    """

    def bind_tools(
        self,
        tools: Sequence[Union[Dict[str, Any], Type[BaseModel], Callable, BaseTool]],
        *,
        tool_choice: Optional[
            Union[Dict[str, str], str]
        ] = None,
        **kwargs: Any,
    ) -> Any:
        """Bind tool-like objects to this chat model.

        Args:
            tools: A list of tool definitions to bind to this chat model.
                Can be a dictionary, pydantic model, callable, or BaseTool. Pydantic
                models, callables, and BaseTools will be automatically converted to
                their schema dictionary representation.
            tool_choice: Which tool to require the model to call.
                Options are:
                    name of the tool (str): calls corresponding tool;
                    "auto" or None: automatically selects a tool (including no tool);
                    "any": force at least one tool to be called;
                    or a dict of the form:
                        {"type": "tool", "name": "tool_name"},
                        or {"type: "any"},
                        or {"type": "auto"};
            **kwargs: Any additional parameters to bind.

        Example:
            from langchain_community.tools.tavily_search import TavilySearchResults
            tool = TavilySearchResults(max_results=2)
            tools = [tool]
            llm_with_tools = WatsonxLLM.bind_tools(tools)
        """
        formatted_tools = [self.convert_to_watson_tool(tool) for tool in tools]
        if not tool_choice:
            pass
        elif isinstance(tool_choice, dict):
            kwargs["tool_choice"] = tool_choice
        elif isinstance(tool_choice, str) and tool_choice in ("any", "auto"):
            kwargs["tool_choice"] = {"type": tool_choice}
        elif isinstance(tool_choice, str):
            kwargs["tool_choice"] = {"type": "tool", "name": tool_choice}
        else:
            raise ValueError(
                f"Unrecognized 'tool_choice' type {tool_choice=}. Expected dict, "
                f"str, or None."
            )
        return self.bind(tools=formatted_tools, **kwargs)

    def convert_to_watson_tool(self, tool: Union[Dict[str, Any], Type[BaseModel], Callable, BaseTool]) -> Dict[str, Any]:
        """Converts various tool types to a Watson-compatible format."""
        if isinstance(tool, BaseModel):
            return tool.schema()
        elif isinstance(tool, BaseTool):
            return tool.to_dict()
        elif callable(tool):
            return self.callable_to_dict(tool)
        elif isinstance(tool, dict):
            return tool
        else:
            raise TypeError(f"Unsupported tool type: {type(tool)}")

    def bind(self, tools: Sequence[Dict[str, Any]], **kwargs: Any) -> Any:
        """Method to actually bind the tools to the model.
        This should be implemented based on the specific needs of WatsonxLLM.
        """
        # Example implementation, should be adapted to the actual WatsonxLLM needs
        logger.info("Binding tools to WatsonxLLM model.")
        # Replace the below line with actual binding logic
        return tools

    def callable_to_dict(self, func: Callable) -> Dict[str, Any]:
        """Convert a callable to a dictionary representation."""
        # Example implementation
        return {
            "name": func.__name__,
            "description": func.__doc__,
            "parameters": [],  # This should be adapted to extract parameters if needed
        }

# Example usage
if __name__ == "__main__":
    from langchain_community.tools.tavily_search import TavilySearchResults
    
    tool = TavilySearchResults(max_results=2)
    tools = [tool]
    
    llm = WatsonxLLM()
    llm_with_tools = llm.bind_tools(tools)

In [ ]:
from langchain_ibm import WatsonxLLM
from typing import Sequence, Union, Dict, Any, Optional, Callable, Type
from langchain_core.pydantic_v1 import BaseModel, Field, SecretStr, root_validator
from langchain_core.tools import BaseTool
from typing import (
    Any,
    Callable,
    Dict,
    Literal,
    Optional,
    Sequence,
    Type,
    Union,
)

class WatsonxLLM:
    # Assuming other parts of WatsonxLLM class are defined elsewhere

    def bind_tools(
        self,
        tools: Sequence[Union[Dict[str, Any], Type[BaseModel], Callable, BaseTool]],
        *,
        tool_choice: Optional[
            Union[Dict[str, str], Literal["any", "auto"], str]
        ] = None,
        **kwargs: Any,
    ) -> Runnable:
        """Bind tool-like objects to this chat model.

        Args:
            tools: A list of tool definitions to bind to this chat model.
                Can be a dictionary, pydantic model, callable, or BaseTool. Pydantic
                models, callables, and BaseTools will be automatically converted to
                their schema dictionary representation.
            tool_choice: Which tool to require the model to call.
                Options are:
                    name of the tool (str): calls corresponding tool;
                    "auto" or None: automatically selects a tool (including no tool);
                    "any": force at least one tool to be called;
                    or a dict of the form:
                        {"type": "tool", "name": "tool_name"},
                        or {"type: "any"},
                        or {"type: "auto"};
            **kwargs: Any additional parameters to bind.

        Example:
            from langchain_community.tools.tavily_search import TavilySearchResults
            tool = TavilySearchResults(max_results=2)
            tools = [tool]
            llm_with_tools = WatsonxLLM.bind_tools(tools)
        """
        formatted_tools = [self.convert_to_watson_tool(tool) for tool in tools]
        if not tool_choice:
            pass
        elif isinstance(tool_choice, dict):
            kwargs["tool_choice"] = tool_choice
        elif isinstance(tool_choice, str) and tool_choice in ("any", "auto"):
            kwargs["tool_choice"] = {"type": tool_choice}
        elif isinstance(tool_choice, str):
            kwargs["tool_choice"] = {"type": "tool", "name": tool_choice}
        else:
            raise ValueError(
                f"Unrecognized 'tool_choice' type {tool_choice=}. Expected dict, "
                f"str, or None."
            )
        return self.bind(tools=formatted_tools, **kwargs)

    def convert_to_watson_tool(self, tool: Union[Dict[str, Any], Type[BaseModel], Callable, BaseTool]) -> Dict[str, Any]:
        """Converts various tool types to a Watson-compatible format."""
        if isinstance(tool, BaseModel):
            return tool.schema()
        elif isinstance(tool, BaseTool):
            return tool.to_dict()
        elif callable(tool):
            return self.callable_to_dict(tool)
        elif isinstance(tool, dict):
            return tool
        else:
            raise TypeError(f"Unsupported tool type: {type(tool)}")
    def bind(self, tools: Sequence[Dict[str, Any]], **kwargs: Any) -> Runnable:
        """Method to actually bind the tools to the model.
        This should be implemented based on the specific needs of WatsonxLLM.
        """
        # Example implementation, should be adapted to the actual WatsonxLLM needs
        pass

    def callable_to_dict(self, func: Callable) -> Dict[str, Any]:
        """Convert a callable to a dictionary representation."""
        # Example implementation
        return {
            "name": func.__name__,
            "description": func.__doc__,
            "parameters": [],  # This should be adapted to extract parameters if needed
        }

In [ ]:
import logging
import os
from typing import Any, Dict, Iterator, List, Mapping, Optional, Union

from ibm_watsonx_ai import Credentials  # type: ignore
from ibm_watsonx_ai.foundation_models import Model, ModelInference  # type: ignore
from langchain_core.callbacks import CallbackManagerForLLMRun
from langchain_core.language_models.llms import BaseLLM
from langchain_core.outputs import Generation, GenerationChunk, LLMResult
from langchain_core.pydantic_v1 import Extra, Field, SecretStr, root_validator
from langchain_core.utils import convert_to_secret_str, get_from_dict_or_env

logger = logging.getLogger(__name__)


class WatsonxLLM(BaseLLM):
    """
    IBM watsonx.ai large language models.

    To use, you should have ``langchain_ibm`` python package installed,
    and the environment variable ``WATSONX_APIKEY`` set with your API key, or pass
    it as a named parameter to the constructor.


    Example:
        .. code-block:: python

            from ibm_watsonx_ai.metanames import GenTextParamsMetaNames
            parameters = {
                GenTextParamsMetaNames.DECODING_METHOD: "sample",
                GenTextParamsMetaNames.MAX_NEW_TOKENS: 100,
                GenTextParamsMetaNames.MIN_NEW_TOKENS: 1,
                GenTextParamsMetaNames.TEMPERATURE: 0.5,
                GenTextParamsMetaNames.TOP_K: 50,
                GenTextParamsMetaNames.TOP_P: 1,
            }

            from langchain_ibm import WatsonxLLM
            watsonx_llm = WatsonxLLM(
                model_id="google/flan-ul2",
                url="https://us-south.ml.cloud.ibm.com",
                apikey="*****",
                project_id="*****",
                params=parameters,
            )
    """

    model_id: str = ""
    """Type of model to use."""

    deployment_id: str = ""
    """Type of deployed model to use."""

    project_id: str = ""
    """ID of the Watson Studio project."""

    space_id: str = ""
    """ID of the Watson Studio space."""

    url: Optional[SecretStr] = None
    """Url to Watson Machine Learning or CPD instance"""

    apikey: Optional[SecretStr] = None
    """Apikey to Watson Machine Learning or CPD instance"""

    token: Optional[SecretStr] = None
    """Token to CPD instance"""

    password: Optional[SecretStr] = None
    """Password to CPD instance"""

    username: Optional[SecretStr] = None
    """Username to CPD instance"""

    instance_id: Optional[SecretStr] = None
    """Instance_id of CPD instance"""

    version: Optional[SecretStr] = None
    """Version of CPD instance"""

    params: Optional[dict] = None
    """Model parameters to use during generate requests."""

    verify: Union[str, bool, None] = None
    """User can pass as verify one of following:
        the path to a CA_BUNDLE file
        the path of directory with certificates of trusted CAs
        True - default path to truststore will be taken
        False - no verification will be made"""

    streaming: bool = False
    """ Whether to stream the results or not. """

    watsonx_model: ModelInference = Field(default=None, exclude=True)  #: :meta private:

    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid

    @classmethod
    def is_lc_serializable(cls) -> bool:
        return False

    @property
    def lc_secrets(self) -> Dict[str, str]:
        """A map of constructor argument names to secret ids.

        For example:
            {
                "url": "WATSONX_URL",
                "apikey": "WATSONX_APIKEY",
                "token": "WATSONX_TOKEN",
                "password": "WATSONX_PASSWORD",
                "username": "WATSONX_USERNAME",
                "instance_id": "WATSONX_INSTANCE_ID",
            }
        """
        return {
            "url": "WATSONX_URL",
            "apikey": "WATSONX_APIKEY",
            "token": "WATSONX_TOKEN",
            "password": "WATSONX_PASSWORD",
            "username": "WATSONX_USERNAME",
            "instance_id": "WATSONX_INSTANCE_ID",
        }

    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        """Validate that credentials and python package exists in environment."""
        if isinstance(values.get("watsonx_model"), (ModelInference, Model)):
            values["model_id"] = getattr(values["watsonx_model"], "model_id")
            values["deployment_id"] = getattr(
                values["watsonx_model"], "deployment_id", ""
            )
            values["project_id"] = getattr(
                getattr(values["watsonx_model"], "_client"),
                "default_project_id",
            )
            values["space_id"] = getattr(
                getattr(values["watsonx_model"], "_client"), "default_space_id"
            )
            values["params"] = getattr(values["watsonx_model"], "params")
        else:
            values["url"] = convert_to_secret_str(
                get_from_dict_or_env(values, "url", "WATSONX_URL")
            )
            if "cloud.ibm.com" in values.get("url", "").get_secret_value():
                values["apikey"] = convert_to_secret_str(
                    get_from_dict_or_env(values, "apikey", "WATSONX_APIKEY")
                )
            else:
                if (
                    not values["token"]
                    and "WATSONX_TOKEN" not in os.environ
                    and not values["password"]
                    and "WATSONX_PASSWORD" not in os.environ
                    and not values["apikey"]
                    and "WATSONX_APIKEY" not in os.environ
                ):
                    raise ValueError(
                        "Did not find 'token', 'password' or 'apikey',"
                        " please add an environment variable"
                        " `WATSONX_TOKEN`, 'WATSONX_PASSWORD' or 'WATSONX_APIKEY' "
                        "which contains it,"
                        " or pass 'token', 'password' or 'apikey'"
                        " as a named parameter."
                    )
                elif values["token"] or "WATSONX_TOKEN" in os.environ:
                    values["token"] = convert_to_secret_str(
                        get_from_dict_or_env(values, "token", "WATSONX_TOKEN")
                    )
                elif values["password"] or "WATSONX_PASSWORD" in os.environ:
                    values["password"] = convert_to_secret_str(
                        get_from_dict_or_env(values, "password", "WATSONX_PASSWORD")
                    )
                    values["username"] = convert_to_secret_str(
                        get_from_dict_or_env(values, "username", "WATSONX_USERNAME")
                    )
                elif values["apikey"] or "WATSONX_APIKEY" in os.environ:
                    values["apikey"] = convert_to_secret_str(
                        get_from_dict_or_env(values, "apikey", "WATSONX_APIKEY")
                    )
                    values["username"] = convert_to_secret_str(
                        get_from_dict_or_env(values, "username", "WATSONX_USERNAME")
                    )
                if not values["instance_id"] or "WATSONX_INSTANCE_ID" not in os.environ:
                    values["instance_id"] = convert_to_secret_str(
                        get_from_dict_or_env(
                            values, "instance_id", "WATSONX_INSTANCE_ID"
                        )
                    )
            credentials = Credentials(
                url=values["url"].get_secret_value() if values["url"] else None,
                api_key=values["apikey"].get_secret_value()
                if values["apikey"]
                else None,
                token=values["token"].get_secret_value() if values["token"] else None,
                password=values["password"].get_secret_value()
                if values["password"]
                else None,
                username=values["username"].get_secret_value()
                if values["username"]
                else None,
                instance_id=values["instance_id"].get_secret_value()
                if values["instance_id"]
                else None,
                version=values["version"].get_secret_value()
                if values["version"]
                else None,
                verify=values["verify"],
            )

            watsonx_model = ModelInference(
                model_id=values["model_id"],
                deployment_id=values["deployment_id"],
                credentials=credentials,
                params=values["params"],
                project_id=values["project_id"],
                space_id=values["space_id"],
            )
            values["watsonx_model"] = watsonx_model

        return values

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {
            "model_id": self.model_id,
            "deployment_id": self.deployment_id,
            "params": self.params,
            "project_id": self.project_id,
            "space_id": self.space_id,
        }

    @property
    def _llm_type(self) -> str:
        """Return type of llm."""
        return "IBM watsonx.ai"

    @staticmethod
    def _extract_token_usage(
        response: Optional[List[Dict[str, Any]]] = None,
    ) -> Dict[str, Any]:
        if response is None:
            return {"generated_token_count": 0, "input_token_count": 0}

        input_token_count = 0
        generated_token_count = 0

        def get_count_value(key: str, result: Dict[str, Any]) -> int:
            return result.get(key, 0) or 0

        for res in response:
            results = res.get("results")
            if results:
                input_token_count += get_count_value("input_token_count", results[0])
                generated_token_count += get_count_value(
                    "generated_token_count", results[0]
                )

        return {
            "generated_token_count": generated_token_count,
            "input_token_count": input_token_count,
        }

    def _get_chat_params(
        self, stop: Optional[List[str]] = None
    ) -> Optional[Dict[str, Any]]:
        params: Optional[Dict[str, Any]] = {**self.params} if self.params else None
        if stop is not None:
            params = (params or {}) | {"stop_sequences": stop}
        return params

    def _create_llm_result(self, response: List[dict]) -> LLMResult:
        """Create the LLMResult from the choices and prompts."""
        generations = []
        for res in response:
            results = res.get("results")
            if results:
                finish_reason = results[0].get("stop_reason")
                gen = Generation(
                    text=results[0].get("generated_text"),
                    generation_info={"finish_reason": finish_reason},
                )
                generations.append([gen])
        final_token_usage = self._extract_token_usage(response)
        llm_output = {
            "token_usage": final_token_usage,
            "model_id": self.model_id,
            "deployment_id": self.deployment_id,
        }
        return LLMResult(generations=generations, llm_output=llm_output)

    def _stream_response_to_generation_chunk(
        self,
        stream_response: Dict[str, Any],
    ) -> GenerationChunk:
        """Convert a stream response to a generation chunk."""
        if not stream_response["results"]:
            return GenerationChunk(text="")
        return GenerationChunk(
            text=stream_response["results"][0]["generated_text"],
            generation_info=dict(
                finish_reason=stream_response["results"][0].get("stop_reason", None),
                llm_output={
                    "model_id": self.model_id,
                    "deployment_id": self.deployment_id,
                },
            ),
        )

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        """Call the IBM watsonx.ai inference endpoint.
        Args:
            prompt: The prompt to pass into the model.
            stop: Optional list of stop words to use when generating.
            run_manager: Optional callback manager.
        Returns:
            The string generated by the model.
        Example:
            .. code-block:: python

                response = watsonx_llm.invoke("What is a molecule")
        """
        result = self._generate(
            prompts=[prompt], stop=stop, run_manager=run_manager, **kwargs
        )
        return result.generations[0][0].text

    def _generate(
        self,
        prompts: List[str],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        stream: Optional[bool] = None,
        **kwargs: Any,
    ) -> LLMResult:
        """Call the IBM watsonx.ai inference endpoint which then generate the response.
        Args:
            prompts: List of strings (prompts) to pass into the model.
            stop: Optional list of stop words to use when generating.
            run_manager: Optional callback manager.
        Returns:
            The full LLMResult output.
        Example:
            .. code-block:: python

                response = watsonx_llm.generate(["What is a molecule"])
        """
        params = self._get_chat_params(stop=stop)
        should_stream = stream if stream is not None else self.streaming
        if should_stream:
            if len(prompts) > 1:
                raise ValueError(
                    f"WatsonxLLM currently only supports single prompt, got {prompts}"
                )
            generation = GenerationChunk(text="")
            stream_iter = self._stream(
                prompts[0], stop=stop, run_manager=run_manager, **kwargs
            )
            for chunk in stream_iter:
                if generation is None:
                    generation = chunk
                else:
                    generation += chunk
            assert generation is not None
            if isinstance(generation.generation_info, dict):
                llm_output = generation.generation_info.pop("llm_output")
                return LLMResult(generations=[[generation]], llm_output=llm_output)
            return LLMResult(generations=[[generation]])
        else:
            response = self.watsonx_model.generate(
                prompt=prompts, params=params, **kwargs
            )
            return self._create_llm_result(response)

    def _stream(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> Iterator[GenerationChunk]:
        """Call the IBM watsonx.ai inference endpoint which then streams the response.
        Args:
            prompt: The prompt to pass into the model.
            stop: Optional list of stop words to use when generating.
            run_manager: Optional callback manager.
        Returns:
            The iterator which yields generation chunks.
        Example:
            .. code-block:: python

                response = watsonx_llm.stream("What is a molecule")
                for chunk in response:
                    print(chunk, end='')
        """
        params = self._get_chat_params(stop=stop)
        for stream_resp in self.watsonx_model.generate_text_stream(
            prompt=prompt, raw_response=True, params=params, **kwargs
        ):
            if not isinstance(stream_resp, dict):
                stream_resp = stream_resp.dict()
            chunk = self._stream_response_to_generation_chunk(stream_resp)

            if run_manager:
                run_manager.on_llm_new_token(chunk.text, chunk=chunk)
            yield chunk

    def get_num_tokens(self, text: str) -> int:
        response = self.watsonx_model.tokenize(text, return_tokens=False)
        return response["result"]["token_count"]

    def get_token_ids(self, text: str) -> List[int]:
        raise NotImplementedError("API does not support returning token ids.")